In [8]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import OneHotEncoder, StandardScaler

%matplotlib inline

In [9]:
def gini(actual, pred, cmpcol = 0, sortcol = 1):
    assert( len(actual) == len(pred) )
    all = np.asarray(np.c_[ actual, pred, np.arange(len(actual)) ], dtype=np.float)
    all = all[ np.lexsort((all[:,2], -1*all[:,1]))]
    totalLosses = all[:,0].sum()
    giniSum = all[:,0].cumsum().sum() / totalLosses
    giniSum -= (len(actual) + 1) / 2.
    return giniSum / len(actual)
    
def gini_normalized(a, p):
    return gini(a, p) / gini(a, a)

In [10]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

target_train = df_train['target'].values
id_test = df_test['id'].values

df_train=df_train.drop(['target','id'],axis=1)
df_test=df_test.drop(['id'], axis = 1)
combine= pd.concat([df_train,df_test],axis=0)

In [11]:
cat_features = [a for a in combine.columns if a.endswith('cat')]
for column in cat_features:
    temp=pd.get_dummies(pd.Series(combine[column]))
    combine=pd.concat([combine,temp],axis=1)
    combine=combine.drop([column],axis=1)

In [ ]:
df_train = df_train.replace(-1, np.NaN)
df_train = df_train.fillna(0)

In [22]:
SS = StandardScaler()

In [ ]:
xtrain, xvalid, ytrain, yvalid = train_test_split(df_train, target_train, test_size=0.3, random_state=13)

xtrain, xvalid = process_cat_features(xtrain, xvalid, cat_features[-2:])

xtrain = SS.fit_transform(xtrain)
xvalid = SS.transform(xvalid)

In [74]:
LR = LogisticRegression(random_state=13, class_weight='balanced')
LR.fit(xtrain, ytrain)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=13,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [75]:
ypred = LR.predict(xvalid)
confusion_matrix(yvalid, ypred)

array([[108175,  63814],
       [  2933,   3642]])

In [76]:
gini_normalized(yvalid, ypred)

0.18515672513939851

In [27]:
sample_submission = pd.read_csv('sample_submission.csv')
sample_submission['target'] = LR.predict_proba(test)[:, 1]
sample_submission.head()
sample_submission.to_csv('pred2.csv', index=False)

ValueError: X has 58 features per sample; expecting 228